# Importing Data

In [10]:
library(haven)
country_level_GPS_data = read_dta("data/GPS_dataset_country_level/country_v11.dta")
head(country_level_GPS_data)

country,isocode,patience,risktaking,posrecip,negrecip,altruism,trust
Afghanistan,AFG,-0.20136012,0.12076426,0.28964090,0.25471246,0.166455463,0.3159644
Algeria,DZA,0.05981525,0.39153048,-0.59825534,0.25490081,-0.009968001,-0.1773706
Argentina,ARG,-0.22930798,0.04150317,0.15967931,-0.14045727,0.027346682,-0.2067333
Australia,AUS,0.65700376,0.13713655,0.06966002,0.02218975,0.155331448,0.2989726
Austria,AUT,0.60828501,-0.06182915,0.16104659,-0.05541545,-0.035363130,0.1546281
Bangladesh,BGD,0.08113671,-0.19806775,0.15436757,0.11328817,0.906463265,-0.1733653


The data is an index among a number of preferences by country.

In [13]:
individual_level_GPS_data = read_dta("data/GPS_dataset_individual_level/individual_v11.dta")
head(individual_level_GPS_data)

country,isocode,ison,region,language,date,id_gallup,wgt,patience,risktaking,posrecip,negrecip,altruism,trust,subj_math_skills
Turkey,TUR,792,Adana,Turkish,2012-03-07,7.1008e+12,0.2717829,0.04717605,1.020203,0.5943841338,-0.3671751,-0.1399527,1.6797541,7
Turkey,TUR,792,Adana,Turkish,2012-03-08,7.1008e+12,0.2717829,-0.67569751,0.387177,0.6622341275,0.0772513,-0.1399527,0.9504338,3
Turkey,TUR,792,Adana,Turkish,2012-03-08,7.1008e+12,0.4422594,0.31825367,1.020203,-0.0009299947,0.0772513,-0.6069670,0.5857736,7
Turkey,TUR,792,Adana,Turkish,2012-03-07,7.1008e+12,1.4236710,0.49897206,1.271528,0.9598911405,0.0772513,0.5605687,0.5857736,7
Turkey,TUR,792,Adana,Turkish,2012-03-07,7.1008e+12,0.7053564,0.58933121,1.122619,1.3253982067,-0.3671751,0.3270616,1.6797541,9
Turkey,TUR,792,Adana,Turkish,2012-03-07,7.1008e+12,2.1355065,-0.58533835,1.963511,1.3253982067,-0.2190329,0.3270616,1.6797541,7
